<a href="https://colab.research.google.com/github/abdulquawiyy-owolabi/Deep-Learning/blob/main/Tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# sset GPU
def set_device():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("Warning: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("Using GPU.")
  return device

In [ ]:
DEVICE = set_device()

Using GPU.


# Implement a Shallow Narrow Linear Neural Network

In [ ]:
class LLNet(nn.Module):
  """
  A Linear Neural Network with one hidden layer.
  """
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(LLNet, self).__init__()
    self.in_hid = nn.Linear(input_dim, hidden_dim, bias=False)
    self.hid_out = nn.Linear(hidden_dim, output_dim, bias=False)

  def forward(self, a):
    hid = self.in_hid(a)  #hidden layer
    # hid = torch.relu(hid)
    out = self.hid_out(hid)   #output layer
    return out, hid

In [ ]:
# Singular decomposition
def net_svd(model, in_dim):
  W_tot = torch.eye(in_dim)
  for weight in model.parameters():
    W_tot = weight.detach() @ W_tot
  U, E, V = torch.svd(W_tot)
  return U, E, V

# representational similarity matrix
def net_rsm(h):
  rsm = h@h.T
  return rsm


In [ ]:
def train(model, inputs, targets, n_epochs, lr, illusory_i=0):
  in_dim = input.size(1)
  losses = np.zeros(n_epochs)
  modes = np.zeros((n_epochs, in_dim))
  rs_mats = []
  illusion = np.zeros(n_epochs)
  optimizer = optim.SGD(model.parameters(), lr=lr)
  criterion = nn.MSELoss()

  for i in range(n_epochs):
    optimizer.zero_grad()
    outputs, hid = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Singular  Value Decomposition
    U, E, V = net_svd(model, in_dim)

    # Calculating representational similarity matrix
    RSM = net_rsm(hid.detach())

    # Network Prediction of Illusory_i for the lst feature
    pred_ij = outputs.detach()[illusory_i]

    # Loggings
    losses[i] = loss.item()
    modes[i] = E.detach().numpy()
    rs_mats.append(RSM.numpy())
    illusion[i] = pred_ij.numpy()

  return losses, modes, np.array(rs_mats), illusion



In [ ]:
def ex_initializer_(model, gamma=1e-12):
  for weight in model.parameters():
    n_out, n_in = weight.shape
    sigma = gamma ** (torch.sqrt(1/(n_in + n_out)))
    nn.init.normal_(weight, mean=0, std=sigma)

In [ ]:
lr = 100.0
gamma = 1e-2  #initialization scale
n_epochs = 1000
dim_input = 8
dim_hidden = 30
dim_output = 10000

# Model Instantiation
dlnn_model = LLNet(dim_input, dim_hidden, dim_output)
ex_initializer_(dlnn_model, gamma)
tree_labels, tree_features = load_tree_data()
input = torch.tensor(tree_features).float()
label_tensor = torch.tensor(tree_labels).float()

# Training
losses, *_ = train(dlnn_model, input, target, n_epochs, lr)